In [2]:
import configparser
import hashlib
import itertools  # for reading the parameters file
import sys  # for system errors and printouts
from pathlib import Path  # for paths of files
import os  # for reading the input data
import time  # for timing
import pandas as pd
import random
import numpy as np

In [3]:
# Global parameters
parameter_file = 'default_parameters.ini'  # the main parameters file
# the main path where all the data directories are
data_main_directory = Path('data')
# dictionary that holds the input parameters, key = parameter name, value = value
parameters_dictionary = dict()
# dictionary of the input documents, key = document id, value = the document
document_list = dict()


# DO NOT CHANGE THIS METHOD
# Reads the parameters of the project from the parameter file 'file'
# and stores them to the parameter dictionary 'parameters_dictionary'
def read_parameters():
    config = configparser.ConfigParser()
    config.read(parameter_file)
    for section in config.sections():
        for key in config[section]:
            if key == 'data':
                parameters_dictionary[key] = config[section][key]
            elif key == 'naive':
                parameters_dictionary[key] = bool(config[section][key])
            elif key == 't':
                parameters_dictionary[key] = float(config[section][key])
            else:
                parameters_dictionary[key] = int(config[section][key])


# DO NOT CHANGE THIS METHOD
# Reads all the documents in the 'data_path' and stores them in the dictionary 'document_list'
def read_data(data_path):
    for (root, dirs, file) in os.walk(data_path):
        for f in file:
            file_path = data_path / f
            doc = open(file_path).read().strip().replace('\n', ' ')
            file_id = int(file_path.stem)
            document_list[file_id] = doc


# DO NOT CHANGE THIS METHOD
# Calculates the Jaccard Similarity between two documents represented as sets
def jaccard(doc1, doc2):
    return len(doc1.intersection(doc2)) / float(len(doc1.union(doc2)))


# DO NOT CHANGE THIS METHOD
# Define a function to map a 2D matrix coordinate into a 1D index.
def get_triangle_index(i, j, length):
    if i == j:  # that's an error.
        sys.stderr.write("Can't access triangle matrix with i == j")
        sys.exit(1)
    if j < i:  # just swap the values.
        temp = i
        i = j
        j = temp

    # Calculate the index within the triangular array. Taken from pg. 211 of:
    # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
    # adapted for a 0-based index.
    k = int(i * (length - (i + 1) / 2.0) + j - i) - 1

    return k


# DO NOT CHANGE THIS METHOD
# Calculates the similarities of all the combinations of documents and returns the similarity triangular matrix
def naive():
    docs_Sets = []  # holds the set of words of each document

    for doc in document_list.values():
        docs_Sets.append(set(doc.split()))

    # Using triangular array to store the similarities, avoiding half size and similarities of i==j
    num_elems = int(len(docs_Sets) * (len(docs_Sets) - 1) / 2)
    similarity_matrix = [0 for x in range(num_elems)]
    for i in range(len(docs_Sets)):
        for j in range(i + 1, len(docs_Sets)):
            similarity_matrix[get_triangle_index(i, j, len(docs_Sets))] = jaccard(
                docs_Sets[i], docs_Sets[j])

    return similarity_matrix


In [4]:
read_parameters()
#print(parameters_dictionary['data'])

# Reading the data
print("Data reading...")
data_folder = data_main_directory / parameters_dictionary['data']
t0 = time.time()
read_data(data_folder)
document_list = {k: document_list[k] for k in sorted(document_list)}
t1 = time.time()
print(len(document_list), "documents were read in", t1 - t0, "sec\n")

# Naive
naive_similarity_matrix = []
if parameters_dictionary['naive']:
    print("Starting to calculate the similarities of documents...")
    t2 = time.time()
    naive_similarity_matrix = naive()
    t3 = time.time()
    print("Calculating the similarities of", len(naive_similarity_matrix),
            "combinations of documents took", t3 - t2, "sec\n")


Data reading...
20 documents were read in 0.1324901580810547 sec

Starting to calculate the similarities of documents...
Calculating the similarities of 190 combinations of documents took 0.004998207092285156 sec



In [5]:
def k_shingles():
    docs_k_shingles = []  # holds the k-shingles of each document
    # implement your code here

    # Get the value k from the parameters dictionary
    k = parameters_dictionary.get("k")

    for key in document_list:
        document = document_list[key]
        words = document.split()

        shingles_in_doc = []
        for i in range(len(words)-k+1):

            shingle = words[i:i + k]

            shingle = ' '.join(shingle)

            if shingle not in shingles_in_doc:
                shingles_in_doc.append(shingle)

        docs_k_shingles.append(shingles_in_doc)

    return docs_k_shingles


docs_k_shingles = k_shingles()

In [6]:
# METHOD FOR TASK 2
# Creates a signatures set of the documents from the k-shingles list
def signature_set(shingles):
    # docs_sig_sets = []
    # print(shingles)

    docs_signature_sets = []
    # print("len", len(document_list))
    for key in range(0, len(document_list)):
        # print("key ", key)
        # print(document_list[key])
        # print(len(document_list))

        shingle = shingles[key]
        # print("shingle", shingle)
        signature_set_shingle = set()

        # print(type(document_list))

        # for shingle in document_list
        # document = document_list[key+1]

        # print("doc: ",document)
        for i in range(len(shingles[key])):
            hash_val = hash(shingle[i])
            # print(hash_val)
            if hash_val not in signature_set_shingle:
                signature_set_shingle.add(hash_val)

            # shing = shingles[key][i]
            # print(shing in document)
        # print(signature_set_shingle)
        docs_signature_sets.append(signature_set_shingle)
    return docs_signature_sets

docs_signature_sets = signature_set(docs_k_shingles)

In [7]:


# Sjekker hvis et tall er et prim tall
def is_prime(n):
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False
    if n < 9:
        return True
    if n % 3 == 0:
        return False
    r = int(n**0.5)
    # since all primes > 3 are of the form 6n ± 1
    # start with f=5 (which is prime)
    # and test f, f+2 for being prime
    # then loop by 6.
    f = 5
    while f <= r:
        if n % f == 0:
            return False
        if n % (f+2) == 0:
            return False
        f += 6
    return True

random.seed(11)

def minHash(document_vector):
    num_permutations = parameters_dictionary.get("permutations")
    num_documents = len(document_list)

    # Nye
    signatures = np.full((num_documents, num_permutations), np.inf)
    #Gammel 
    #signatures = np.full((num_permutations,num_documents), np.inf)
    
    count = 0
    perms = 0
    docies = 0

    #Nye
    for i in range(num_documents):
    #Gammel
    #for i in range(num_permutations):
        a = random.randint(1, 400)
        b = random.randint(1, 400)
        cont = True
        while cont:
            # Generate a random number in the range [lower_bound, upper_bound]
            p = random.randint(400, 1000)

            # Check if the number is prime
            if is_prime(p):
                cont = False
        perms += 1
        #nye
        for j in range(num_permutations):
        #gammel
        #for j in range(num_documents):
            docies += 1
            localcount = 1
            for sig in document_vector[j]:

                hash_value = (a * sig + b) % p
                count += 1
                localcount += 1

                if hash_value < signatures[i][j]:
                    signatures[i][j] = hash_value
    return signatures

signatures = minHash(docs_signature_sets)


In [15]:
#print(signaturesGammel)
#print(signaturesNy)
print(signatures)

[[ 2.  1.  9. 12. 12. 10.  1.  0.  1.  2.]
 [ 1.  2. 10.  0.  0.  2.  2.  0.  5.  3.]
 [ 0.  4.  0.  4.  3.  6.  0.  1.  1. 12.]
 [ 3.  2.  1.  1.  6. 19.  0.  7.  0.  1.]
 [ 5.  1.  3.  5.  5. 11.  0.  0.  6.  6.]
 [ 2.  2.  3.  0.  0.  1. 12.  0.  7.  3.]
 [ 3.  2.  3.  0.  0.  0.  3.  2.  2.  0.]
 [ 0.  0.  2.  0.  1.  3.  0.  2.  6.  0.]
 [ 1.  0.  1.  1.  1.  0.  0.  2.  1.  0.]
 [ 0.  1.  0.  0.  4.  2.  2.  0.  2.  0.]
 [ 6.  1.  1.  2.  0.  3.  0.  1.  0.  4.]
 [ 0.  6.  6.  0.  0.  2.  0.  0.  5.  0.]
 [ 1.  0.  1.  1.  0. 14.  0.  0.  1.  2.]
 [ 1.  1.  1.  1.  1.  2.  1.  0.  0.  2.]
 [ 0.  3.  7.  2.  4.  3.  1.  5.  0.  4.]
 [ 1.  1.  7.  0.  3.  4.  0.  0.  1.  5.]
 [ 0.  1.  6.  9.  8. 13.  3.  2.  0.  0.]
 [ 0.  1.  1.  0.  1.  0.  0.  1.  1.  2.]
 [ 2.  0.  1.  1.  0.  3.  0.  0.  0.  2.]
 [ 3.  2.  2.  3.  0.  0.  1.  0.  2.  7.]]


In [9]:
""" 
#print(signaturesNy)
templis =[]
tempcols = []
for i in signaturesNy:
    templis.append(i[0])
for j in signaturesGammel:
    tempcols.append(j[0])
print(templis)
print(tempcols)
"""

' \n#print(signaturesNy)\ntemplis =[]\ntempcols = []\nfor i in signaturesNy:\n    templis.append(i[0])\nfor j in signaturesGammel:\n    tempcols.append(j[0])\nprint(templis)\nprint(tempcols)\n'

In [18]:
# METHOD FOR TASK 4
# Hashes the MinHash Signature Matrix into buckets and find candidate similar documents
def lsh(m_matrix):

    candidates = []  # list of candidate sets of documents for checking similarity
    #print(m_matrix.T)
    #m_matrix = m_matrix.T
    # implement your code here
    buckets = parameters_dictionary.get("buckets")
    # permutations = parameters_dictionary.get("permutations")
    # bands = len(m_matrix[0])/buckets
    # print(bands)
    # for signature in m_matrix:
    bucket_dict = {}
    for i in range(buckets):
        bucket_dict[i] = set()

    #print("shape ",m_matrix.shape[0])
    # Hash each column of the signature matrix into a bucket
    for j in range(m_matrix.shape[0]):
        #hash_val = hash(tuple(m_matrix[:, j]))
        hash_val = hash(tuple(m_matrix[j, :]))
        bucket_idx = hash_val % buckets
        bucket_dict[bucket_idx].add(j)

    # Find candidate similar documents from the buckets
    candidates = set()
    for bucket in bucket_dict.values():
        #print(len(bucket))
        if len(bucket) > 1:
            pairs = list(itertools.combinations(bucket, 2))
            #print(pairs)
            candidates.update(pairs)

    return candidates

In [20]:
candidates = lsh(signatures)


In [22]:

print(candidates)
print(signatures)



{(3, 7), (4, 12), (17, 3), (8, 18), (19, 15), (10, 12), (1, 9), (0, 11), (18, 13), (4, 11), (10, 11), (0, 4), (0, 10), (4, 10), (17, 7), (5, 14), (8, 13), (0, 12), (11, 12), (2, 6), (16, 5), (16, 14)}
[[ 2.  1.  9. 12. 12. 10.  1.  0.  1.  2.]
 [ 1.  2. 10.  0.  0.  2.  2.  0.  5.  3.]
 [ 0.  4.  0.  4.  3.  6.  0.  1.  1. 12.]
 [ 3.  2.  1.  1.  6. 19.  0.  7.  0.  1.]
 [ 5.  1.  3.  5.  5. 11.  0.  0.  6.  6.]
 [ 2.  2.  3.  0.  0.  1. 12.  0.  7.  3.]
 [ 3.  2.  3.  0.  0.  0.  3.  2.  2.  0.]
 [ 0.  0.  2.  0.  1.  3.  0.  2.  6.  0.]
 [ 1.  0.  1.  1.  1.  0.  0.  2.  1.  0.]
 [ 0.  1.  0.  0.  4.  2.  2.  0.  2.  0.]
 [ 6.  1.  1.  2.  0.  3.  0.  1.  0.  4.]
 [ 0.  6.  6.  0.  0.  2.  0.  0.  5.  0.]
 [ 1.  0.  1.  1.  0. 14.  0.  0.  1.  2.]
 [ 1.  1.  1.  1.  1.  2.  1.  0.  0.  2.]
 [ 0.  3.  7.  2.  4.  3.  1.  5.  0.  4.]
 [ 1.  1.  7.  0.  3.  4.  0.  0.  1.  5.]
 [ 0.  1.  6.  9.  8. 13.  3.  2.  0.  0.]
 [ 0.  1.  1.  0.  1.  0.  0.  1.  1.  2.]
 [ 2.  0.  1.  1.  0.  3.

In [32]:
print(list(candidates)[0][0])
pair = 
i = 0
jac = jaccard(set(signatures[list(candidates)[i][0]]),set(signatures[list(candidates)[i][1]]))
print(jac)
#print(signatures[0])


3
0.7142857142857143


In [46]:
""" similarity(d1, d2) = #(hi(d1) == hi(d2))
permutations
"""
# METHOD FOR TASK 5
# Calculates the similarities of the candidate documents
def candidates_similarities(candidate_docs, min_hash_matrix):
    similarity_matrix = []
    #print("candidates ", candidate_docs)
    #print("\n\n Min hash", min_hash_matrix)
    #jaccard(doc1, doc2)
    for pair in candidate_docs:  
            
        index1 = pair[0] 
        index2 = pair[1] 
        jac = jaccard(set(min_hash_matrix[index1]),set(min_hash_matrix[index2]))
        #print("jac ", jac)
        similarity_matrix.append([index1, index2, jac])

    """
    similarity(d1, d2) = # (hi(d1)) == hi(d2)) / permutations
    similarity(d1, d2) = antall ord i d1 og i d2 /permutations


    """

    # implement your code here
    return similarity_matrix

sim_matrix = candidates_similarities(candidates, signatures)


In [47]:
print(sim_matrix)

[[3, 7, 0.7142857142857143], [4, 12, 0.25], [17, 3, 0.42857142857142855], [8, 18, 0.75], [19, 15, 0.5714285714285714], [10, 12, 0.42857142857142855], [1, 9, 0.42857142857142855], [0, 11, 0.25], [18, 13, 0.75], [4, 11, 0.42857142857142855], [10, 11, 0.42857142857142855], [0, 4, 0.2], [0, 10, 0.3333333333333333], [4, 10, 0.5], [17, 7, 0.6], [5, 14, 0.625], [8, 13, 1.0], [0, 12, 0.42857142857142855], [11, 12, 0.3333333333333333], [2, 6, 0.2857142857142857], [16, 5, 0.4], [16, 14, 0.36363636363636365]]


In [54]:
# METHOD FOR TASK 6
# Returns the document pairs of over t% similarity
#sim_matrix

"""
After calculating the similarity for the candidate document pairs from the previous
task, count how many document pairs have similarity over the threshold input parameter t and print their ids. Also, count how many of the pairs are false positive and
false negative using the naive method for the accurate similarity.
"""
def return_results(lsh_similarity_matrix):
    document_pairs = []
    t = parameters_dictionary['t']
    count = 0
    for pair in lsh_similarity_matrix:
        threshold = pair[2]
        if threshold > t:
            count += 1
            id1 = pair[0]
            id2 = pair[1]
            print("Id1: ", id1, " Id2: ", id2, " similarity: ", threshold)
            document_pairs.append([id1, id2])
        #print(pair[2])

    # implement your code here

    print("Total above threshold ", t, " :", count)
    return document_pairs


return_results(sim_matrix)

Id1:  3  Id2:  7  similarity:  0.7142857142857143
Id1:  8  Id2:  18  similarity:  0.75
Id1:  18  Id2:  13  similarity:  0.75
Id1:  5  Id2:  14  similarity:  0.625
Id1:  8  Id2:  13  similarity:  1.0
Total above threshold  0.6  =  5


[[3, 7], [8, 18], [18, 13], [5, 14], [8, 13]]

In [ ]:

# METHOD FOR TASK 6
def count_false_neg_and_pos(lsh_similarity_matrix, naive_similarity_matrix):
    false_negatives = 0
    false_positives = 0

    # implement your code here

    return false_negatives, false_positives